In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import csv
from openpyxl import Workbook
import os

In [2]:
def moneycontrol(mc_url):
    # Fetch the Moneycontrol data
    mc_data = urlopen(mc_url)
    mc_html = mc_data.read()

    # Parse the HTML
    mc_soup = soup(mc_html, 'html.parser')

    # Extract column titles
    headers = mc_soup.findAll('th')

    column_titles = [ct.text for ct in headers]
    column_titles = column_titles[:6]
    # column_titles[6] = column_titles[6][0:17]
    ## Upto Done with Headers

    # Extract company names
    span = mc_soup.find_all('span', class_='gld13')
    company_span_list = [spani.text[:-37] for spani in span]
    # Save company names to a CSV file
    filename = 'mc_NIFTY_500_company.csv'
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        company_name = '\n'.join(company_span_list)
        f.write(company_name)
        f.close()

    ## Now It's time for Valuable Digits
    # First of all we extract the elements which is not neccessory for final data

    # Remove unnecessary HTML elements
    # Remove p elements
    tdrs = mc_soup.find_all('p')
    for re in tdrs:
        re.decompose()
    # Remove strongs elements
    tdrst = mc_soup.find_all('strong')
    for re in tdrs:
        re.decompose()
    tws = mc_soup.find_all('div', {'class': 'title2'})
    for re in tws:
        re.decompose()
    tws = mc_soup.find_all('td', {'class': 'vol'})
    for re in tws:
        re.decompose()
    tws = mc_soup.find_all('td', {'class': 'del'})
    for re in tws:
        re.decompose()
    twsas = mc_soup.find_all('td', {'width': '300'})
    for re in twsas:
        re.decompose()

    # Extract valuable data
    ## td_data is Our final Valuable Data
    td_data = mc_soup.find_all('td', {'align': 'right'})

    # Final digit List
    ## Little bit of data cleansing :)
    digit_list = []
    for tt in td_data[0:2505]:
        x = tt.text
        x = x.replace(",", "")
        digit_list.append(x)

    # Save data to a CSV file
    with open('mc_NIFTY_digits.csv', 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        #      writer.writerow(column_titles[1:])
        for i in range(0, 2505, 5):
            writer.writerow([digit_list[i], digit_list[i + 1], digit_list[i + 2], digit_list[i + 3], digit_list[i + 4]])
        f.close()

    # Clear entire csv file
    with open('final_mc_NIFTY_list.csv', 'w+', encoding='utf-8') as f:
        f.truncate()
        f.close
    #Merge data and company names into a final CSV file
    with open('mc_NIFTY_digits.csv', 'r', encoding='utf-8') as read_temp, open('mc_NIFTY_500_company.csv', 'r',encoding='utf-8') as header, open(
            'final_mc_NIFTY_list.csv', 'a', encoding='utf-8', newline='') as final_list:
        reader = csv.reader(read_temp)
        writer = csv.reader(header)
        final_obj = csv.writer(final_list)

        final_obj.writerow(column_titles)  # Put Headings on top of list
        for a, b in zip(writer, reader):
            final_obj.writerow(a + b)  # write 50 rows (Company_name+ value)
        read_temp.close()
        header.close()
        final_list.close()

    ## Generate Excel file
    # Read the CSV file
    csv_filename = 'final_mc_NIFTY_list.csv'
    xlsx_filename = 'NIFTY_500_sheet.xlsx'
    # Create a new Excel workbook
    workbook = Workbook()
    worksheet = workbook.active
    # Open the CSV file and write its contents to the Excel worksheet
    with open(csv_filename, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            worksheet.append(row)
    # Save the Excel workbook
    workbook.save(xlsx_filename)
    print(f'{xlsx_filename} has been created successfully!')


    # Now remove the unnnecessory CSV files
    os.remove('mc_NIFTY_500_company.csv')
    os.remove('mc_NIFTY_digits.csv')
    print('SpreadSheet is Ready for U!')

In [3]:
if __name__ == '__main__':
    mc_url = "https://www.moneycontrol.com/stocks/marketstats/nse-mostactive-stocks/nifty-500-7/"
    moneycontrol(mc_url)


NIFTY_500_sheet.xlsx has been created successfully!
SpreadSheet is Ready for U!


In [12]:
import pandas as pd
df = pd.read_csv('final_mc_NIFTY_list.csv')
df

,Company Name,High,Low,Last Price,% Chg,Value (Rs. cr.)
0,HDFC Bank,1564.40,1523.75,1529.65,-1.53,4719.30
1,Reliance,2382.45,2350.00,2354.95,-0.42,2258.02
2,Canara Bank,386.00,368.50,382.95,5.08,1256.18
3,ICICI Bank,960.90,947.40,950.30,-0.94,1207.06
4,SJVN,74.10,69.35,72.25,1.69,1152.08
...,...,...,...,...,...,...
496,Aether Ind,992.45,975.95,977.60,-0.82,2.57
497,Ingersoll Rand,3056.55,2966.00,2986.70,-1.66,2.40
498,TTK Prestige,788.50,771.85,778.50,-0.35,2.38
499,Galaxy Surfacta,2663.70,2601.00,2606.60,-1.67,2.20
